In [9]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
import os
import tensorflow as tf
from IPython import display
from sklearn import metrics
import glob
import sys

# Import training data and preprocess

In [42]:
dota2_df = None
for filename in glob.glob(os.path.join('dota2_training_data','*.csv')):
    if type(dota2_df)==None:
        dota2_df = pd.read_csv(filename, usecols=range(1,13))
    else:
        dota2_df = pd.concat([dota2_df, pd.read_csv(filename, usecols=range(1,13))],ignore_index=True)

dota2_df = dota2_df.reindex(np.random.permutation(dota2_df.index))
dota2_df.describe()

,match_id,radiant_win,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,8.797000e+03,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000,8797.000000
mean,4.253016e+09,0.549733,50.808116,51.427759,52.109924,51.585313,51.716153,51.900307,51.776401,51.577924,50.882346,51.083665
std,1.790517e+06,0.497549,34.429014,34.492457,34.563422,34.404611,34.813327,34.561568,34.140065,34.651073,34.428431,34.512934
min,4.252092e+09,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.252121e+09,0.000000,21.000000,21.000000,22.000000,22.000000,21.000000,22.000000,22.000000,22.000000,21.000000,21.000000
50%,4.252128e+09,1.000000,44.000000,45.000000,46.000000,44.000000,45.000000,47.000000,46.000000,45.000000,44.000000,44.000000
75%,4.252140e+09,1.000000,81.000000,82.000000,84.000000,83.000000,84.000000,84.000000,82.000000,84.000000,81.000000,82.000000
max,4.256625e+09,1.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000


In [43]:
def preprocess_features(dota2_df):
    """Take dota2_df and create a dataframe containing only the features for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
    """
    
    #Use arrays of heroes for each team as features rather than each hero
    #individually to help model fit?
    #This is an attempt to make the model more similar to the movie review
    #text analysis example in the Google ML Crash Course
    processed_df = pd.DataFrame()
    
    processed_df['radiant_heroes'] = list(np.array(dota2_df.loc[:,['radiant_heroes0',
                                                'radiant_heroes1','radiant_heroes2',
                                                'radiant_heroes3','radiant_heroes4']]))
    processed_df['dire_heroes'] = list(np.array(dota2_df.loc[:,['dire_heroes0',
                                    'dire_heroes1','dire_heroes2',
                                    'dire_heroes3','dire_heroes4']]))
    
    ##only features (to start with) are the heroes in the game
    #processed_df = dota2_df[['radiant_heroes0', 'radiant_heroes1',
    #                   'radiant_heroes2', 'radiant_heroes3', 'radiant_heroes4',
    #                    'dire_heroes0', 'dire_heroes1', 'dire_heroes2',
    #                    'dire_heroes3', 'dire_heroes4']]
    
    ##create two synthetic features that is the product of all heroes for each team
    #processed_df['radiant_hero_product'] = dota2_df['radiant_heroes0']*dota2_df['radiant_heroes1']*dota2_df['radiant_heroes2']dota2_df['radiant_heroes3']*dota2_df['radiant_heroes4']
    #processed_df['dire_hero_product'] = dota2_df['dire_heroes0']*dota2_df['dire_heroes1']*dota2_df['dire_heroes2']dota2_df['dire_heroes3']*dota2_df['dire_heroes4']
    
    return processed_df
    
def preprocess_targets(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    
    return target_df

In [46]:
#Split training and validation datasets with a given training fraction. 

#to begin with we have a sample of 7050 matches to validation the model with.
#This may not be enough games to accurately determine win probability but probably good enough for first validations

training_fraction = 0.7
head_num = int(training_fraction * len(dota2_df))
tail_num = len(dota2_df)-head_num

training_examples = preprocess_features(dota2_df.head(head_num))
training_targets = preprocess_targets(dota2_df.head(head_num))

validation_examples = preprocess_features(dota2_df.tail(tail_num))
validation_targets = preprocess_targets(dota2_df.tail(tail_num))

#print("Training features summary")
#training_examples.style
print("\nTraining targets summary")
display.display(training_targets.describe())

#print("\nvalidation feature summary")
#validation_examples.style
print("\nvalidation target summary")
display.display(validation_targets.describe())


Training targets summary


,radiant_win
count,6157.000000
mean,0.549943
std,0.497540
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000



validation target summary


,radiant_win
count,2640.000000
mean,0.549242
std,0.497664
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


# Build TFRecords file
Do this to nicely handle the features that is a numpy array (radiant_heroes and dire_heroes)

In [47]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [48]:
def convert_to_tfrecords(filename, examples, targets):
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(filename)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    target_array = np.array(targets['radiant_win'])
    
    for i in range(len(radiant_hero_array[:])):
        # print how many images are saved every 1000 images
        if not i % 1000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the image
        radiant_heroes = radiant_hero_array[:][i].tostring()
        dire_heroes = dire_hero_array[:][i].tostring()
        target = target_array[i]
        
        # Create a feature
        feature = {'train/radiant_heroes': _bytes_feature(tf.compat.as_bytes(radiant_heroes)),
                   'train/dire_heroes': _bytes_feature(tf.compat.as_bytes(dire_heroes)),
                   'train/targets': _int64_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [51]:
convert_to_tfrecords(os.path.join('dota2_training_data', 'dota2_training_data.tfrecords'),
                     preprocess_features(dota2_df),
                     preprocess_targets(dota2_df))


Train data: 0/8797
Train data: 1000/8797
Train data: 2000/8797
Train data: 3000/8797
Train data: 4000/8797
Train data: 5000/8797
Train data: 6000/8797
Train data: 7000/8797
Train data: 8000/8797


# Build input pipeline

In [39]:
def _parse_function(record):
    """Extracts features and labels.
  
    Args:
        record: File path to a TFRecord file    
      Returns:
    A `tuple` `(labels, features)`:
      features: A dict of tensors representing the features
      labels: A tensor with the corresponding labels.
    """
    features = {
        "train/radiant_heroes": tf.FixedLenFeature([], dtype=tf.string), #array of five 8 bit ints for heroes
        "train/dire_heroes": tf.FixedLenFeature([], dtype=tf.string), #array of five 8-bit ints for heroes
        "train/targets": tf.FixedLenFeature(shape=[1], dtype=tf.int64)
                }
  
    parsed_features = tf.parse_single_example(record, features)
  
    radiant_heroes = tf.decode_raw(parsed_features['train/radiant_heroes'], tf.int64)
    dire_heroes = tf.decode_raw(parsed_features['train/dire_heroes'], tf.int64)
    radiant_win = parsed_features['train/targets']

    return  {'radiant_heroes':radiant_heroes, 'dire_heroes':dire_heroes}, radiant_win

#### Check the parse function worked

In [40]:
# Create the Dataset object.
ds = tf.data.TFRecordDataset(os.path.join('dota2_training_data','match_ids_4252145739-4252126461.tfrecords'))
# Map features and labels with the parse function.
ds = ds.map(_parse_function)

ds

<MapDataset shapes: ({radiant_heroes: (?,), dire_heroes: (?,)}, (1,)), types: ({radiant_heroes: tf.int64, dire_heroes: tf.int64}, tf.int64)>

In [41]:
n = ds.make_one_shot_iterator().get_next()
sess = tf.Session()
sess.run(n)

({'dire_heroes': array([33, 97, 11,  6,  2]),
  'radiant_heroes': array([ 36,  41,  69, 105,  10])},
 array([0]))

# Build Functions for doing linear classifier modelling

In [76]:
def construct_hero_categorical_columns(input_features):
    """Construct categorical features for all features
        Args: 
            input_features: names of input feature columns to use
        returns:
            a set of feature columns
    """
    vocab = list(range(105))+list(range(106,114))+list(range(119,121))
    return set([tf.feature_column.categorical_column_with_vocabulary_list(my_feature,
                                                                          vocabulary_list = vocab, num_oov_buckets =1)
                                                                for my_feature in input_features])

In [77]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a logistic regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = tf.data.Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
        ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [71]:
def train_linear_model(
    learning_rate,
    steps,
    batch_size,
    feature_columns,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    """Trains a linear regression model.
  
    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.
      
    Args:
        learning_rate: A `float`, the learning rate.
        steps: A non-zero `int`, the total number of training steps. A training step
          consists of a forward and backward pass using a single batch.
        feature_columns: A `set` specifying the input feature columns to use.
        training_examples: A `DataFrame` containing one or more columns from
          `dota2_df` to use as input features for training.
        training_targets: A `DataFrame` containing exactly one column from
          `dota2_df` to use as target for training.
      
    Returns:
        A `LinearClassifier` object trained on the training data.
    """

    periods = 20
    steps_per_period = steps / periods

    # Create a linear regressor object.
    my_optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    linear_classifier = tf.estimator.LinearClassifier(
      feature_columns=feature_columns,
      optimizer=my_optimizer,
      config=tf.estimator.RunConfig(keep_checkpoint_max=1)
      )
  
    training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["radiant_win"], 
                                          batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["radiant_win"], 
                                          num_epochs=1, 
                                          shuffle=False)
    
    validation_input_fn = lambda: my_input_fn(validation_examples, 
                                          validation_targets["radiant_win"], 
                                          batch_size=batch_size)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                          validation_targets["radiant_win"], 
                                          num_epochs=1, 
                                          shuffle=False)
    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("LogLoss error (on validation data):")

    training_errors = []
    validation_errors = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        linear_classifier.train(
            input_fn=training_input_fn,
            steps=steps_per_period
            )
        # Take a break and compute predictions.
        training_predictions = list(linear_classifier.predict(input_fn=predict_training_input_fn))
        training_probabilities = np.array([item['probabilities'] for item in training_predictions])
        training_pred_class_id = np.array([item['class_ids'][0] for item in training_predictions])
        training_pred_one_hot = tf.keras.utils.to_categorical(training_pred_class_id,2)
            
        validation_predictions = list(linear_classifier.predict(input_fn=predict_validation_input_fn))
        validation_probabilities = np.array([item['probabilities'] for item in validation_predictions])    
        validation_pred_class_id = np.array([item['class_ids'][0] for item in validation_predictions])
        validation_pred_one_hot = tf.keras.utils.to_categorical(validation_pred_class_id,2)    
        print("Model training finished.")
        # Compute training and validation errors.
        training_log_loss = metrics.log_loss(training_targets, training_pred_one_hot)
        validation_log_loss = metrics.log_loss(validation_targets, validation_pred_one_hot)
        # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, validation_log_loss))
        # Add the loss metrics from this period to our list.
        training_errors.append(training_log_loss)
        validation_errors.append(validation_log_loss)
    print("Model training finished.")
    
    # Remove event files to save disk space.
    _ = map(os.remove, glob.glob(os.path.join(linear_classifier.model_dir, 'events.out.tfevents*')))
  
    # Calculate final predictions (not probabilities, as above).
    final_predictions = linear_classifier.predict(input_fn=predict_validation_input_fn)
    final_predictions = np.array([item['class_ids'][0] for item in final_predictions])
  
  
    accuracy = metrics.accuracy_score(validation_targets, final_predictions)
    print("Final accuracy (on validation data): %0.2f" % accuracy)

    # Output a graph of loss metrics over periods.
    plt.ylabel("LogLoss")
    plt.xlabel("Periods")
    plt.title("LogLoss vs. Periods")
    plt.plot(training_errors, label="training")
    plt.plot(validation_errors, label="validation")
    plt.legend()
    plt.show()
  
    # Output a plot of the confusion matrix.
    cm = metrics.confusion_matrix(validation_targets, final_predictions)
    # Normalize the confusion matrix by row (i.e by the number of samples
    # in each class).
    cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    ax = sns.heatmap(cm_normalized, cmap="bone_r")
    ax.set_aspect(1)
    plt.title("Confusion matrix")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()

    return linear_classifier

In [72]:
trained_linear_model = train_linear_model(
    learning_rate=0.01,
    steps=500,
    batch_size=50,
    feature_columns=construct_hero_categorical_columns(training_examples),
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/2_/ffhj27wn1bbgt7ldy1qll5600000gn/T/tmpjm0y4te1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x16fd17f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
LogLoss error (on validation data):


TypeError: Expected binary or unicode string, got array([33, 97, 11,  6,  2])

In [86]:
def train_dnn_model(
    learning_rate,
    regularization,
    hidden_units,
    steps,
    batch_size,
    feature_columns,
    training_examples,
    training_targets):
    """Trains a deep neural network regression model.
  
    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.
      
    Args:
        learning_rate: A `float`, the learning rate.
        l1_regularization: A `float`, the L1 regularization rate. This regularizatin 
          penalizes non-zero weights in the model to prevent overfitting as well as
          save memory.
        l2_regularization: A `float`, the L2 regularization rate. This regularization
          penalizes large weights in the model to prevent overfitting.
        hidden_units: A list of `ints`, the number of nodes in each layer for the DNN
          model.
        steps: A non-zero `int`, the total number of training steps. A training step
          consists of a forward and backward pass using a single batch.
        feature_columns: A `set` specifying the input feature columns to use.
        training_examples: A `DataFrame` containing one or more columns from
          `dota2_df` to use as input features for training.
        training_targets: A `DataFrame` containing exactly one column from
          `dota2_df` to use as target for training.
      
    Returns:
        A `DNNClassifer` object trained on the training data.
    """

    periods = 10
    steps_per_period = steps / periods

    # Create a linear regressor object.
    my_optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    dnn_classifier = tf.estimator.DNNClassifier(
      feature_columns=feature_columns,
      hidden_units=hidden_units,
      optimizer=my_optimizer
      )
  
    training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["radiant_win"], 
                                          batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["radiant_win"], 
                                                  num_epochs=1, 
                                                  shuffle=False)

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("RMSE (on training data):")
    training_rmse = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        dnn_classifier.train(
            input_fn=training_input_fn,
            steps=steps_per_period
            )
        # Take a break and compute predictions.
        training_predictions = dnn_classifier.predict(input_fn=predict_training_input_fn)
    
    print("Model training finished.")


    return dnn_classifier

In [ ]:
trained_linear_model = train_dnn_model(
    learning_rate=0.07,
    regularization=0.01
    steps=1000,
    batch_size=100,
    feature_columns=construct_hero_categorical_columns(training_examples),
    training_examples=training_examples,
    training_targets=training_targets)